# Task 1
"Generate adversarial examples in the context of the zero-knowledge threat model."

In [1]:
import os
import sys
from pathlib import Path

In [2]:
# Set up proper directory paths

project_path = Path().absolute().parent
src_path = project_path.joinpath("src")

# Ensure the paths are properly assigned
# If this assertion fails, change project_dir as needed to become the project directory
# If project_dir is correct, change the name in the assertion check
assert project_path.name == "project-athena", "Parent directory name assertion failed (check the path)"

In [3]:
# Add src_dir to module paths
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

In [4]:
import time

import numpy as np

In [5]:
from matplotlib import pyplot as plt

In [6]:
from attacks.attack import generate
from models.athena import ENSEMBLE_STRATEGY, Ensemble
from utils.data import subsampling
from utils.file import load_from_json, dump_to_json
from utils.metrics import error_rate, get_corrections
from utils.model import load_lenet, load_pool

ModuleNotFoundError: No module named 'art'

In [ ]:
def generate_ae(model, data, labels, attack_configs, save=False, output_dir=None, device=None):
    """
    Generate adversarial examples for a model
    """
    
    print("Generating adversarial examples")
    
    ae_files = []
    
    num_attacks = attack_configs.get("num_attacks")
    data_loader = (data, labels)
    
    # For plotting
    img_rows, img_cols = data.shape[1], data.shape[2]

    if len(labels.shape) > 1:
        labels = np.asarray([np.argmax(p) for p in labels])

    # Generate attacks
    for attack_id in range(num_attacks):
        key = f"configs{attack_id}"
        
        data_adversarial = generate(model=model,
                                    data_loader=data_loader,
                                    attack_args=attack_configs[key],
                                    device=device)
        
        # Evaluate adversarial examples
        y_pred = model.predict(data_adversarial)
        y_pred = np.asarray([np.argmax(p) for p in y_pred])

        err = error_rate(y_pred=y_pred, y_true=labels)
        
        print(f">>> error rate: {err}")

        # Save adversarial examples
        if save:
            if output_dir is None:
                raise ValueError("Cannot save images to a none path.")
            
            file_name = f"task1-{attack_configs[key]['description']}-{time.monotonic()}.npy"
            file_path = Path(output_dir).joinpath(file_name)
            np.save(file_path, data_adversarial)
            
            ae_files.append(file_name)
            
            print(f"Saved adversarial example to file [{file_path}]")
        
        print((attack_configs[key]['attack']))
        
        batch_size = min(data.shape[0], 2)
        fig = plt.figure(figsize=(1, batch_size), dpi=300)
        
        for i in range(batch_size):
            fig.add_subplot(1, batch_size, i + 1)
            plt.axis("off")
            plt.imshow(data_adversarial[i].reshape((img_rows, img_cols)), cmap='gray')
        
        
        plt.show()
        plt.savefig('results.pdf')
    
    print("Done generating adversarial examples")
    
    return ae_files

In [ ]:
def evaluate(trans_configs, model_configs, data_configs,
             save=False, output_dir=None):
    """
    Apply transformation(s) on images.
    :param trans_configs: dictionary. The collection of the parameterized transformations to test.
        in the form of
        { configsx: {
            param: value,
            }
        }
        The key of a configuration is 'configs'x, where 'x' is the id of corresponding weak defense.
    :param model_configs:  dictionary. Defines model related information.
        Such as, location, the undefended model, the file format, etc.
    :param data_configs: dictionary. Defines data related information.
        Such as, location, the file for the true labels, the file for the benign samples,
        the files for the adversarial examples, etc.
    :param save: boolean. Save the transformed sample or not.
    :param output_dir: path or str. The location to store the transformed samples.
        It cannot be None when save is True.
    :return:
    """
    
    # Load baseline defense (PGD-ADT model)
    baseline = load_lenet(file=model_configs['pgd_trained'],
                          trans_configs=None,
                          use_logits=False,
                          wrap=False)

    # Load undefended model (UM)
    file = project_path.joinpath(model_configs['dir'], model_configs['um_file'])
    undefended = load_lenet(file=file,
                            trans_configs=trans_configs.get('configs0'),
                            wrap=True)
    
    print(f">>> UM: {type(undefended)}")

    # Load weak defenses into a pool
    pool, _ = load_pool(trans_configs=trans_configs,
                        model_configs=model_configs,
                        active_list=True,
                        wrap=True)
    
    # Create an AVEP ensemble from the WD pool
    wds = list(pool.values())
    ensemble = Ensemble(classifiers=wds, strategy=ENSEMBLE_STRATEGY.AVEP.value)
    
    print(f">>> wds: {type(wds)} {type(wds[0])}")

    # Load benign samples
    bs_file = project_path.joinpath(data_configs['dir'], data_configs['bs_file'])
    
    # Hacky workaround for benign data being in different directory as adversarial data
    if 'benign_dir' in data_configs:
        bs_file = project_path.joinpath(data_configs['benign_dir'], data_configs['bs_file'])
    
    x_bs = np.load(bs_file)
    img_rows, img_cols = x_bs.shape[1], x_bs.shape[2]

    # Load true labels
    label_file = project_path.joinpath(data_configs['dir'], data_configs['label_file'])
    labels = np.load(label_file)

    print(f">>> Evaluating UM on [{bs_file}]")
    
    # Get indices of benign samples that are correctly classified by the targeted model
    pred_bs = undefended.predict(x_bs)
    corrections = get_corrections(y_pred=pred_bs, y_true=labels)

    # Evaluate adversarial examples
    results = {}
    
    # Only run on one example
    ae_files = data_configs.get('ae_files')
    ae_file = project_path.joinpath(data_configs['dir'], ae_files[4])
    
    for file in ae_files:
        results[file] = {}
        
        ae_file = project_path.joinpath(data_configs['dir'], file)
        x_adversarial = np.load(ae_file)
        
        print(f">>> Running evaluations on [{ae_file}]")

        print(f">>> Evaluating UM")

        # Evaluate undefended model on adversarial examples
        pred = undefended.predict(x_adversarial)
        err = error_rate(y_pred=pred, y_true=labels, correct_on_bs=corrections)
        results[file]['UM'] = err

        print(f">>> Evaluating ensemble")

        # Evaluate ensemble model on adversarial examples
        pred = ensemble.predict(x_adversarial)
        err = error_rate(y_pred=pred, y_true=labels, correct_on_bs=corrections)
        results[file]['Ensemble'] = err
        
        print(f">>> Evaluating baseline model")

        # Evaluate baseline model on adversarial examples
        pred = baseline.predict(x_adversarial)
        err = error_rate(y_pred=pred, y_true=labels, correct_on_bs=corrections)
        results[file]['PGD-ADT'] = err

        print(f">>> Evaluations on [{ae_file}]:\n{results[file]}")
    
    result_file = project_path.joinpath("results/ae_evaluation_results.json")
    dump_to_json(results, result_file)
    
    print(f">>> Evaluations on all ae_files dumped to [{result_file}]")

In [16]:
# Load data configs
file = src_path.joinpath("configs/demo/data-mnist.json")
data_configs = load_from_json(file)

# Load model configs
model_configs = load_from_json(src_path.joinpath("configs/demo/model-mnist.json"))
attack_configs = load_from_json(src_path.joinpath("configs/demo/attack-zk-mnist.json"))

output_path = project_path.joinpath("data")

# Fix configs
# i.e. remove relative paths
data_configs['dir'] = str(project_path.joinpath("data"))
data_configs['sub_dir'] = str(project_path.joinpath("results"))

model_configs['dir'] = str(project_path.joinpath("models/cnn"))
model_configs['pgd_trained'] = str(project_path.joinpath("models/baseline", Path(model_configs['pgd_trained']).name))

NameError: name 'load_from_json' is not defined

In [15]:
# Load benign samples
file = project_path.joinpath(data_configs['dir'], data_configs['bs_file'])
X_bs = np.load(file)

NameError: name 'data_configs' is not defined

In [14]:
# Load true labels
file =  project_path.joinpath(data_configs['dir'], data_configs['label_file'])
labels = np.load(file)

NameError: name 'data_configs' is not defined

In [13]:
# Load model
model_file = project_path.joinpath(model_configs['dir'], model_configs['um_file'])
target = load_lenet(file=model_file, wrap=True)

NameError: name 'model_configs' is not defined

In [10]:
# Load the benign samples
# TODO: Is this redundant duplicate code?
data_file = project_path.joinpath(data_configs['dir'], data_configs['bs_file'])
data_bs = np.load(data_file)

NameError: name 'data_configs' is not defined

In [11]:
# Load true labels
label_file = project_path.joinpath(data_configs['dir'], data_configs['label_file'])
labels = np.load(label_file)

NameError: name 'data_configs' is not defined

In [12]:
# Remove files in data with prefix "task1"

for f in os.listdir(output_path):
    if f.startswith("task1"):
        os.remove(output_path.joinpath(f))
        
        print(f"Removed {f}")

NameError: name 'output_path' is not defined

# Generating the adverserial examples

This section of the code is where the adverserial examples are generated. The team has decided to use the Fast Gradient Sign Method(FGSM), Project Gradient Descent(PGD), and Basic Iteractive Method(BIM) attacks to create the adversary examples. Each of the methods were given four variants each with different values to see how much the original image is perturbed per point value and how it affects the error rate of the model.

#Fast Gradient Sign Method



#Project Gradient Descent

#Basic Iteractive Method

In [7]:
# Generate adversarial examples
ae_files = generate_ae(model=target, data=data_bs,labels=labels, attack_configs=attack_configs,
            save=True, output_dir=output_path)

NameError: name 'generate_ae' is not defined

In [8]:
# Clear results folder
result_files = os.listdir(project_path.joinpath("results"))

for f in result_files:
    file = project_path.joinpath("results", f)
    
    try:
        os.remove(file)
    except Exception:
        print(f"Failed to remove {f}")

print("Cleared results folder")

Failed to remove .ipynb_checkpoints
Failed to remove evaluation
Cleared results folder


In [9]:
# Replace adversarial examples with the newly generated examples
data_configs['ae_files'] = ae_files

NameError: name 'ae_files' is not defined

In [10]:
# Load configs
file = src_path.joinpath("configs/demo/athena-mnist.json")
trans_configs = load_from_json(file)

output_dir = project_path.joinpath("data")
output_dir.mkdir(exist_ok=True)

# Evaluate model
evaluate(trans_configs=trans_configs,
         model_configs=model_configs,
         data_configs=data_configs,
         save=False,
         output_dir=output_dir)

NameError: name 'load_from_json' is not defined